# Recommender Systems
## Writing A Machine Learning Algorithm for Recommending Movies

In dit script probeer ik een algoritme te schrijven die de beoordeling van films gaat voorspellen voor mensen. Recommender Systems worden gebruikt door bedrijven zoals YouTube en Netflix om jou langer op hun platform te houden. Ook kan het gebruikt worden door online kledingwinkels die willen voorspellen welke kleding jij leuk vindt.

De dataset die wordt gebruikt is de MovieLens database. Deze is te vinden op [grouplens.org/](https://grouplens.org/datasets/movielens/). Daarnaast volg ik een stappenplan die is beschreven in [towardsdatascience.com](https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d.)

In [2]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Het inladen van de dataset. Kies voor de kleine of de grote dataset aan de hand van je doel. 
# De grote dataset duurt lang om in te laden, maar zal wel betere resultaten leveren
BigDataSet = False
DataSet = "data" if BigDataSet else "data_small"
df = pd.read_csv(DataSet + '/ratings.csv', sep=',', names=['userID','movieID','rating','titmestamp'])
df.head()

,userID,movieID,rating,titmestamp
0,userId,movieId,rating,timestamp
1,1,1,4.0,964982703
2,1,3,4.0,964981247
3,1,6,4.0,964982224
4,1,47,5.0,964983815


Door de movies.csv dataset samen te voegen met de ratings.csv kunnen we de ratings koppelen aan een film

In [18]:
movieTitles = pd.read_csv(DataSet + '/movies.csv', names=['movieID','title','genres'])
df = pd.merge(df, movieTitles, on='movieID')
df.head()

,userID,movieID,rating,titmestamp,title,genres
0,userId,movieId,rating,timestamp,title,genres
1,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Dit is wat elke kolom betekent:
- `userID` is de ID van elke gebruiker. 
- `movieID` is de ID van elke film. 
- `rating` is de beoordeling van de gebruiker `userID` op film `movieID`. Dit kan een waarde zijn tussen 1 en 5
- `timestamp` is de tijd dat de film beoordeeld was
- `title` is de naam van de film en uit welk jaar die kwam
- `genres` zijn de verschillende soorten genres van de film

Met de commando's `describe` krijgen we een samenvatting van onze dataset. 

In [22]:
df.describe(include='all')

,userID,movieID,rating,titmestamp,title,genres
count,100837,100837,100837,100837,100837,100837
unique,611,9725,11,85044,9720,952
top,414,356,4.0,1459787998,Forrest Gump (1994),Comedy
freq,2698,329,26818,128,329,7196


We zien wat interessante dingen. De dataset heeft 100837 beoordelingen met 611 unieke gebruikers en 9725 unieke films. Daarnaast is de meest beoordeelde film Forrest Gump uit 1994. Deze film is 329 keer beoordeeld. 